# 🎬 VideoRobot — One-Click Colab Runner
این نوت‌بوک ریپوی VideoRobot را کلون کرده، FFmpeg و وابستگی‌ها را نصب می‌کند، بک‌اند را روی Colab GPU بالا می‌آورد، سلامت را بررسی می‌کند، دارایی‌ها را از Google Drive می‌خواند، یک مانفیست نمونه می‌سازد، و ویدیو نهایی را دانلود می‌کند.

In [ ]:
# 🧠 Mount Google Drive for assets (optional)
from google.colab import drive
drive.mount('/content/drive')
print('✅ Google Drive mounted. Put assets under drive/MyDrive/videorobot_assets if needed.')

In [ ]:
# 📦 Clone repository fresh
import shutil, pathlib, subprocess
REPO_URL = "https://github.com/englishpodcasteasy-glitch/videorobot.git"
WORK = pathlib.Path('/content/videorobot')
shutil.rmtree(WORK, ignore_errors=True)
subprocess.run(['git', 'clone', REPO_URL, str(WORK)], check=True)
print('✅ Repo cloned at', WORK)

In [ ]:
# ⚙️ Install FFmpeg and Python deps
!apt-get update -qq && apt-get install -y ffmpeg
!pip install -q -r /content/videorobot/backend/requirements.txt

In [ ]:
# 🚀 Start backend on port 8000 and check health
import os, time, requests, subprocess, sys
os.environ['BACKEND_PORT'] = os.environ.get('BACKEND_PORT', '8000')
backend = subprocess.Popen([sys.executable, 'backend/main.py'], cwd='/content/videorobot')
base = f"http://127.0.0.1:{os.environ['BACKEND_PORT']}"
for i in range(120):
    try:
        r = requests.get(base + '/healthz', timeout=2)
        if r.ok:
            print('✅ Backend healthy:', r.text)
            break
    except Exception:
        pass
    time.sleep(1)
else:
    raise SystemExit('❌ Backend did not become healthy in time')

In [ ]:
# 📝 Create a sample manifest
import json, pathlib
manifest = {
  "seed": 7,
  "video": {"width": 720, "height": 1280, "fps": 30, "bg_color": "#101318"},
  "tracks": [
    {"type": "text", "content": "سلام دنیا", "start": 0.2, "duration": 3, "x": 40, "y": 80, "size": 72, "color": "#FFFFFF"}
  ]
}
path = pathlib.Path('/content/manifest.json')
path.write_text(json.dumps(manifest, ensure_ascii=False))
print('✅ Manifest saved to', path)

In [ ]:
# 🎥 Submit render job and download output
import urllib.request
base = f"http://127.0.0.1:{os.environ['BACKEND_PORT']}"
resp = requests.post(base + '/render', json=json.loads(open('/content/manifest.json','r',encoding='utf-8').read()), timeout=60)
print('POST /render:', resp.status_code, resp.text)
resp.raise_for_status()
job_id = resp.json().get('job_id')
if not job_id:
    raise SystemExit('❌ No job_id returned')
print('🎬 Job ID:', job_id)
for i in range(600):
    r = requests.get(base + f'/progress/{job_id}', timeout=10)
    if r.ok:
        js = r.json()
        print(f"tick {i}: {js.get('state')} {js.get('pct')}")
        if js.get('state') == 'success':
            break
        if js.get('state') == 'error':
            raise SystemExit(js)
    time.sleep(1)
out = f"/content/{job_id}.mp4"
urllib.request.urlretrieve(base + f'/download?jobId={job_id}', out)
print('✅ Downloaded video:', out)
print('🎉 Done! Check /content or Drive for output.')